In [1]:
%matplotlib notebook
import matplotlib.pyplot as plt
from urllib.request import urlopen
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

In [2]:
data = pd.read_csv('mlb_data/master2016.csv')

In [3]:
data.head()

,Batter,Batter Batting Order,Batter Team,Pitcher Team,Batter Salary,Batter DK Points,Batter Home/Away,Date,Pitcher,Pitcher Batting Order,...,Quality Start Rate,Strikeout-to-Walk Ratio,Strikeouts Per 9 IP,Walks Per 9 IP,Home Runs Allowed Per 9 IP,Pitcher Batting Average On Balls In Play,Strand Rate,Fastball Velocity (MPH),Earned Run Average,Fielding Independent Pitching
0,Troy Tulowitzki,5.0,TOR,TAM,3.7,18.0,0,403,Chris Archer,0.0,...,57.6,3.48,10.42,3.00,1.34,0.314,72.7,94.3,4.02,3.85
1,Francisco Cervelli,5.0,PIT,STL,3.0,15.0,1,403,Adam Wainwright,9.0,...,54.5,2.73,7.29,2.67,1.00,0.332,68.9,90.3,4.62,3.94
2,Gregory Polanco,6.0,PIT,STL,4.0,14.0,1,403,Adam Wainwright,9.0,...,54.5,2.73,7.29,2.67,1.00,0.332,68.9,90.3,4.62,3.94
3,Corey Dickerson,4.0,TAM,TOR,3.5,14.0,1,403,Marcus Stroman,0.0,...,56.3,3.07,7.32,2.38,0.93,0.315,67.8,92.4,4.37,3.73
4,Edwin Encarnacion,4.0,TOR,TAM,4.4,12.0,0,403,Chris Archer,0.0,...,57.6,3.48,10.42,3.00,1.34,0.314,72.7,94.3,4.02,3.85


In [4]:
data = data.loc[:,~data.columns.duplicated()]

In [5]:
data['Batter Team'].unique()

array(['TOR', 'PIT', 'TAM', 'NYM', 'KAN', 'STL', 'COL', 'SFO', 'WAS',
       'LAD', 'CHC', 'MIL', 'ATL', 'CHW', 'BAL', 'CIN', 'PHI', 'SEA',
       'MIN', 'OAK', 'TEX', 'SDG', 'LAA', 'ARI', 'HOU', 'DET', 'MIA',
       'BOS', 'NYY', 'CLE'], dtype=object)

In [6]:
park_factors = pd.read_csv('mlb_data/park_factors/espn2016_park_factors.csv')


In [8]:
park_factors['HOME TEAM'] = [
    'COL',
    'ARI',
    'CLE',
    'BOS',
    'KAN',
    'TOR',
    'TEX',
    'ATL',
    'MIN',
    'NYY',
    'DET',
    'SDG',
    'SFO',
    'PIT',
    'CIN',
    'NYM',
    'MIL',
    'WAS',
    'BAL',
    'SEA',
    'CHW',
    'STL',
    'LAA',
    'TAM',
    'CHC',
    'PHI',
    'MIA',
    'OAK',
    'LAD',
    'HOU'
]

In [9]:
park_factors

,PARK NAME,RUNS,HR,H,2B,3B,BB,HOME TEAM
0,"Coors Field (Denver, Colorado)",1.368,1.265,1.231,1.405,1.390,1.078,COL
1,"Chase Field (Phoenix, Arizona)",1.225,1.292,1.139,1.145,2.032,1.042,ARI
2,"Progressive Field (Cleveland, Ohio)",1.207,1.168,1.115,1.299,0.507,1.119,CLE
3,"Fenway Park (Boston, Massachusetts)",1.199,1.065,1.161,1.424,1.667,0.977,BOS
4,"Kauffman Stadium (Kansas City, Missouri)",1.171,0.783,1.092,1.256,1.571,1.039,KAN
5,"Rogers Centre (Toronto, Ontario)",1.156,1.010,1.083,1.300,1.100,1.039,TOR
6,"Globe Life Park in Arlington (Arlington, Texas)",1.156,1.049,1.100,1.052,1.813,1.029,TEX
7,"Turner Field (Atlanta, Georgia)",1.059,0.770,1.009,0.900,0.936,1.135,ATL
8,"Target Field (Minneapolis, Minnesota)",1.044,1.014,1.064,1.081,1.031,0.875,MIN
9,"Yankee Stadium (New York, New York)",1.035,1.377,0.982,0.871,0.500,1.033,NYY


In [10]:
data.dtypes

Batter                                       object
Batter Batting Order                        float64
Batter Team                                  object
Pitcher Team                                 object
Batter Salary                               float64
Batter DK Points                            float64
Batter Home/Away                              int64
Date                                          int64
Pitcher                                      object
Pitcher Batting Order                       float64
Pitcher Opposing Team                        object
Pitcher Salary                              float64
Pitcher DK Points                           float64
Batter Team.1                                object
Batter Position                              object
Batter Games                                float64
At Bats                                     float64
Pitches Per Plate Appearance                float64
Bunts                                       float64
Grounded Int

In [11]:
data['Home Team'] = data.apply(lambda row: row["Batter Team"] if row['Batter Home/Away'] else row['Pitcher Team'], axis=1)

In [12]:
data.head()

,Batter,Batter Batting Order,Batter Team,Pitcher Team,Batter Salary,Batter DK Points,Batter Home/Away,Date,Pitcher,Pitcher Batting Order,...,Strikeout-to-Walk Ratio,Strikeouts Per 9 IP,Walks Per 9 IP,Home Runs Allowed Per 9 IP,Pitcher Batting Average On Balls In Play,Strand Rate,Fastball Velocity (MPH),Earned Run Average,Fielding Independent Pitching,Home Team
0,Troy Tulowitzki,5.0,TOR,TAM,3.7,18.0,0,403,Chris Archer,0.0,...,3.48,10.42,3.00,1.34,0.314,72.7,94.3,4.02,3.85,TAM
1,Francisco Cervelli,5.0,PIT,STL,3.0,15.0,1,403,Adam Wainwright,9.0,...,2.73,7.29,2.67,1.00,0.332,68.9,90.3,4.62,3.94,PIT
2,Gregory Polanco,6.0,PIT,STL,4.0,14.0,1,403,Adam Wainwright,9.0,...,2.73,7.29,2.67,1.00,0.332,68.9,90.3,4.62,3.94,PIT
3,Corey Dickerson,4.0,TAM,TOR,3.5,14.0,1,403,Marcus Stroman,0.0,...,3.07,7.32,2.38,0.93,0.315,67.8,92.4,4.37,3.73,TAM
4,Edwin Encarnacion,4.0,TOR,TAM,4.4,12.0,0,403,Chris Archer,0.0,...,3.48,10.42,3.00,1.34,0.314,72.7,94.3,4.02,3.85,TAM


In [14]:
data_with_pf = data.merge(park_factors, left_on='Home Team', right_on="HOME TEAM", how='left')

In [16]:
data_with_pf.drop('HOME TEAM', axis=1, inplace=True)

In [17]:
data_with_pf.to_csv('master2016.csv', index=False)

# Trying a model with Park Factors included 

In [18]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.cross_validation import train_test_split

In [19]:
df = data_with_pf

In [21]:
df.columns

Index(['Batter', 'Batter Batting Order', 'Batter Team', 'Pitcher Team',
       'Batter Salary', 'Batter DK Points', 'Batter Home/Away', 'Date',
       'Pitcher', 'Pitcher Batting Order', 'Pitcher Opposing Team',
       'Pitcher Salary', 'Pitcher DK Points', 'Batter Team.1',
       'Batter Position', 'Batter Games', 'At Bats',
       'Pitches Per Plate Appearance', 'Bunts', 'Grounded Into Double Play',
       'Intentional Walks', 'Walks', 'Strikeouts', 'Walk Rate',
       'Walk-To-Strikeout Ratio', 'Contact Rate', 'Stolen Base Opportunity',
       'Batting Average', 'Batting Average on Balls In Play', 'Bunt Rate',
       'Ground Into Double Play Rate', 'Intentional Walk Rate',
       'Strikeout Rate', 'Games', 'Innings Pitched', 'Games Started',
       'Quality Start Rate', 'Strikeout-to-Walk Ratio', 'Strikeouts Per 9 IP',
       'Walks Per 9 IP', 'Home Runs Allowed Per 9 IP',
       'Pitcher Batting Average On Balls In Play', 'Strand Rate',
       'Fastball Velocity (MPH)', 'Earned Run

In [28]:
drop_cols = ['Batter', 'Batter Team', 'Pitcher Team', 'Batter DK Points', 
             'Date', 'Pitcher', 'Pitcher Batting Order', 'Pitcher Opposing Team', 
            'Pitcher DK Points', 'Batter Position', 'Batter Games', 'Home Team', 'PARK NAME', 'Batter Team.1']
X = df.drop(drop_cols, axis=1)

In [29]:
X.columns

Index(['Batter Batting Order', 'Batter Salary', 'Batter Home/Away',
       'Pitcher Salary', 'At Bats', 'Pitches Per Plate Appearance', 'Bunts',
       'Grounded Into Double Play', 'Intentional Walks', 'Walks', 'Strikeouts',
       'Walk Rate', 'Walk-To-Strikeout Ratio', 'Contact Rate',
       'Stolen Base Opportunity', 'Batting Average',
       'Batting Average on Balls In Play', 'Bunt Rate',
       'Ground Into Double Play Rate', 'Intentional Walk Rate',
       'Strikeout Rate', 'Games', 'Innings Pitched', 'Games Started',
       'Quality Start Rate', 'Strikeout-to-Walk Ratio', 'Strikeouts Per 9 IP',
       'Walks Per 9 IP', 'Home Runs Allowed Per 9 IP',
       'Pitcher Batting Average On Balls In Play', 'Strand Rate',
       'Fastball Velocity (MPH)', 'Earned Run Average',
       'Fielding Independent Pitching', 'RUNS', 'HR', 'H', '2B', '3B', 'BB'],
      dtype='object')

In [30]:
y = df['Batter DK Points']

In [31]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [57]:
model = RandomForestRegressor(n_estimators=300)

In [58]:
model.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=300, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [59]:
predictions = model.predict(X_test)
prediction_df = pd.DataFrame({'y': y_test, 'y_hat': predictions})

In [60]:
prediction_df

,y,y_hat
26839,5.0,7.383333
16310,6.0,5.650000
18140,0.0,5.770000
11856,0.0,5.256667
17485,16.0,11.203333
28115,22.0,6.310000
10306,9.0,7.170000
9648,15.0,7.236667
1235,21.0,12.653333
11662,17.0,5.813333


In [61]:
avg_accuracy = np.mean(np.abs((prediction_df.y - prediction_df.y_hat)/prediction_df.y_hat))

In [62]:
avg_accuracy

0.7482492057905417

In [63]:
avg_error = np.mean(np.abs((prediction_df.y - prediction_df.y_hat)))

In [64]:
avg_error

5.511830380467872

In [67]:
lower_end_error = prediction_df.apply(lambda row: row.y - row.y_hat if row.y < row.y_hat else 0, axis=1)

In [68]:
np.mean(abs(lower_end_error))

2.955588327342488

In [65]:
model

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=300, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [69]:
from sklearn.externals import joblib

In [75]:
joblib.dump(model, 'models/14mar2016/model.pkl') 

['models/14mar2016/model.pkl',
 'models/14mar2016/model.pkl_01.npy',
 'models/14mar2016/model.pkl_02.npy',
 'models/14mar2016/model.pkl_03.npy',
 'models/14mar2016/model.pkl_04.npy',
 'models/14mar2016/model.pkl_05.npy',
 'models/14mar2016/model.pkl_06.npy',
 'models/14mar2016/model.pkl_07.npy',
 'models/14mar2016/model.pkl_08.npy',
 'models/14mar2016/model.pkl_09.npy',
 'models/14mar2016/model.pkl_10.npy',
 'models/14mar2016/model.pkl_11.npy',
 'models/14mar2016/model.pkl_12.npy',
 'models/14mar2016/model.pkl_13.npy',
 'models/14mar2016/model.pkl_14.npy',
 'models/14mar2016/model.pkl_15.npy',
 'models/14mar2016/model.pkl_16.npy',
 'models/14mar2016/model.pkl_17.npy',
 'models/14mar2016/model.pkl_18.npy',
 'models/14mar2016/model.pkl_19.npy',
 'models/14mar2016/model.pkl_20.npy',
 'models/14mar2016/model.pkl_21.npy',
 'models/14mar2016/model.pkl_22.npy',
 'models/14mar2016/model.pkl_23.npy',
 'models/14mar2016/model.pkl_24.npy',
 'models/14mar2016/model.pkl_25.npy',
 'models/14mar2016/

In [72]:
model_import = joblib.load('models/14mar2016/model.pkl') 

In [74]:
sum(model_import.predict(X_test) == predictions) == len(predictions)

True